In [ ]:
import os, json
from google import genai
from google.genai import types
import re

# === paths + setup ===
meta_root = "/home/jiaji02/Desktop/EchoBreaker/echo-breaker/survey_metadata"
client = genai.Client(api_key="")

def extract_choice(text, n_opts):
    """extract an integer 1..n_opts from model text"""
    m = re.search(r"\b(\d+)\b", text)
    if m:
        num = int(m.group(1))
        if 1 <= num <= n_opts:
            return num
    return None


for root, dirs, files in os.walk(meta_root):
    # skip folders without metadata
    if "ground_truth.json" not in files:
        continue

    json_path = os.path.join(root, "ground_truth.json")
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    question = data.get("question", "").strip()
    prompt_text = data.get("prompt", "").strip()
    responses = [r["response"] for r in data.get("responses", []) if r.get("response")]
    num_opts = len(responses)
    # build response options string like:
    # "1. Option A\n2. Option B\n3. Option C\n"
    numbered_opts = "".join([f"{i+1}. {opt}; " for i, opt in enumerate(responses)])
    num_range = f"(1-{len(responses)})"

    # find all PNGs in this folder (tweets)
    tweet_images = [f for f in files if f.lower().endswith(".png")]
    if not tweet_images:
        continue

    print(f"\n=== Processing {os.path.basename(root)} ===")

    for img_file in tweet_images:
        img_path = os.path.join(root, img_file)

        # load image bytes
        with open(img_path, "rb") as f:
            img_bytes = f.read()

        # construct LLM prompt
        final_prompt = f"""
            You are analyzing the ideological stance expressed in an online post (image attached).

            Survey question:
            "{question}", 
            "{prompt_text}"

            Response options:
            {numbered_opts}

            Task:
            Based on the content and tone of the attached image, determine which response option {num_range}
            best reflects the stance expressed.
            Respond only with the option number {num_range}, without explanation.
            """
        # print(final_prompt)
        print(f"→ Sending {img_file} to Gemini...")

        try:
            response = client.models.generate_content(
                model="gemini-2.5-flash",
                contents=[
                    types.Part.from_bytes(data=img_bytes, mime_type="image/png"),
                    final_prompt,
                ],
            )
            model_text = response.text.strip()
            print(f"  🧠 Gemini response: {model_text}")

            choice = extract_choice(model_text, num_opts)
            if choice is None:
                print(f"  ⚠️ Could not parse a valid choice from response.")
                continue

            chosen = responses[choice - 1]
            dem_val = float(data['responses'][choice-1]['dem'])
            rep_val = float(data['responses'][choice-1]['rep'])

            result_path = os.path.join(root, img_file.replace(".png", ".json"))
            with open(result_path, "w", encoding="utf-8") as outf:
                json.dump({"dem": dem_val, "rep": rep_val}, outf, indent=2)

            print(f"  ✅ saved → {result_path}")

        except Exception as e:
            print(f"  ❌ error on {img_file}: {e}")


=== Processing bbf626a7-a904-11e1-9412-005056900141_res_q24 ===
→ Sending tweet_1.png to Gemini...
  🧠 Gemini response: 2
  ✅ saved → /home/jiaji02/Desktop/EchoBreaker/echo-breaker/survey_metadata/bbf626a7-a904-11e1-9412-005056900141_res_q24/tweet_1.json
→ Sending tweet_3.png to Gemini...
  🧠 Gemini response: 4
  ✅ saved → /home/jiaji02/Desktop/EchoBreaker/echo-breaker/survey_metadata/bbf626a7-a904-11e1-9412-005056900141_res_q24/tweet_3.json
→ Sending tweet_2.png to Gemini...
  🧠 Gemini response: 1
  ✅ saved → /home/jiaji02/Desktop/EchoBreaker/echo-breaker/survey_metadata/bbf626a7-a904-11e1-9412-005056900141_res_q24/tweet_2.json

=== Processing bb73fe89-a904-11e1-9412-005056900141_res_q16 ===
→ Sending tweet_1.png to Gemini...
  🧠 Gemini response: 5
  ✅ saved → /home/jiaji02/Desktop/EchoBreaker/echo-breaker/survey_metadata/bb73fe89-a904-11e1-9412-005056900141_res_q16/tweet_1.json
→ Sending tweet_3.png to Gemini...
  🧠 Gemini response: 5
  ✅ saved → /home/jiaji02/Desktop/EchoBreaker/ec